In [19]:
import requests
import hashlib
import time
import json
import time
import whisper
import os
import subprocess
from pathlib import Path
import sys
import tempfile


In [20]:
API_KEY = "4YCF385ZNXLEHADRUSFV"
API_SECRET = "YcjAfd9x7S$$qX^4u#n$rvLD$X6c^pT87ShWvF3V"

In [11]:
# Generate the timestamp
timestamp = str(int(time.time()))
auth_hash = hashlib.sha1((API_KEY + API_SECRET + timestamp).encode('utf-8')).hexdigest()

headers = {
    "X-Auth-Date": timestamp,
    "X-Auth-Key": API_KEY,
    "Authorization": auth_hash,
    "User-Agent": "PodcastIndex-Client"
}

# Step 1: Get Trending Podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
params = {"max": 50}  # Get up to 200 trending podcasts

response = requests.get(url, headers=headers, params=params)
trending_podcasts = response.json().get("feeds", [])

print(f"Retrieved {len(trending_podcasts)} trending podcasts.")

Retrieved 50 trending podcasts.


In [12]:

# Step 1: Download trending podcasts
url = "https://api.podcastindex.org/api/1.0/podcasts/trending"
response = requests.get(url, headers=headers, params=params)
trending_data = response.json()

# Step 2: Extract relevant fields
trending_data = response.json()
trending_podcasts = trending_data.get("feeds", [])

simplified_podcasts = [
    {
        "title": feed.get("title"),
        "url": feed.get("url"),
        "description": feed.get("description"),
        "id": feed.get("id"),
        "author": feed.get("author"),
    }
    for feed in trending_podcasts
]


# Step 3: Save to JSON file
output_path = "top_podcasts.json"
with open(output_path, "w") as f:
    json.dump(simplified_podcasts, f, indent=4)

print(f"Saved {len(simplified_podcasts)} podcasts to {output_path}")




Saved 50 podcasts to top_podcasts.json


In [13]:
with open("top_podcasts.json", "r") as f:
    podcasts = json.load(f)

all_episodes = []

for podcast in podcasts:
    feed_id = podcast.get("id")
    podcast_title = podcast.get("title")

    print(f"Fetching episodes for: {podcast_title}")

    response = requests.get(
        "https://api.podcastindex.org/api/1.0/episodes/byfeedid",
        headers=headers,
        params= {"id": feed_id, "max": 1} #more episodes from one podcast - change max
    )

    if response.status_code == 200:
        episodes = response.json().get("items", [])
        for ep in episodes:
            all_episodes.append({
                "podcast": podcast_title,
                "episode_title": ep.get("title"),
                "audio_url": ep.get("enclosureUrl"),
                "id": ep.get("id")
            })
    

# Save to top_episodes.json
with open("top_episodes.json", "w") as f:
    json.dump(all_episodes, f, indent=4)

print(f"✅ Saved {len(all_episodes)} episodes to top_episodes.json")


Fetching episodes for: Your Personal Revolution
Fetching episodes for: Ahi Va
Fetching episodes for: 7 Good Minutes
Fetching episodes for: G.I.O. Get It On
Fetching episodes for: Géopolitique
Fetching episodes for: Devos
Fetching episodes for: WDR Hörspiel-Speicher
Fetching episodes for: To The Contrary with Charlie Sykes
Fetching episodes for: Habits of A Goddess
Fetching episodes for: Regard sur la semaine
Fetching episodes for: Monday devotion
Fetching episodes for: The Eden Podcast with Bruce C. E. Fleming
Fetching episodes for: Cult of Conspiracy
Fetching episodes for: Doc Malik
Fetching episodes for: In Right Relationship
Fetching episodes for: Ask Dr. Drew
Fetching episodes for: Kitap Okumaları
Fetching episodes for: The Dig Bible Podcast
Fetching episodes for: Experiencia Kidzania Monterrey
Fetching episodes for: Aeon Byte Gnostic Radio
Fetching episodes for: North Shore Nine: A Pittsburgh Pirates Podcast
Fetching episodes for: Aeon Byte Inner Sanctum of Gnosis
Fetching episode

In [23]:


def download_audio(url, output_path):
    subprocess.run([
        "yt-dlp",
        "-x", "--audio-format", "mp3",
        "-o", str(output_path),
        url
    ], check=True)

def transcribe_from_url(audio_path):
    try:
        # Download audio into memory
        response = requests.get(audio_url)
        audio_bytes = io.BytesIO(response.content)

        # Load and convert audio to 16kHz mono with pydub
        audio = AudioSegment.from_file(audio_bytes)
        audio = audio.set_frame_rate(16000).set_channels(1)

        # Export as wav to in-memory file
        wav_bytes = io.BytesIO()
        audio.export(wav_bytes, format="wav")
        wav_bytes.seek(0)

        # Load audio for Whisper
        audio_tensor = whisper.audio.load_audio(wav_bytes)
        audio_tensor = whisper.audio.pad_or_trim(audio_tensor)
        mel = whisper.log_mel_spectrogram(audio_tensor).to(model.device)

        # Decode with Whisper
        options = whisper.DecodingOptions()
        result = whisper.decode(model, mel, options)
        return result.text

    except Exception as e:
        print(f"Error processing {audio_url}: {e}")
        return None


In [24]:
# Load Whisper model once
model = whisper.load_model("tiny")  # or "base", "small", etc.

# Directories
AUDIO_DIR = Path("audio")
TRANSCRIPT_DIR = Path("transcripts")
os.makedirs(AUDIO_DIR, exist_ok=True)
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)

# Load episode list
with open("top_episodes.json", "r", encoding="utf-8") as f:
    episodes = json.load(f)

for episode in episodes:
    podcast_title = episode.get("podcast", "unknown_podcast")
    episode_title = episode.get("episode_title", "unknown_episode")
    audio_url = episode.get("audio_url")

    if not audio_url or not audio_url.strip():
        print(f" No valid audio URL for: {episode_title}")
        continue

    print(f"\n Processing: {podcast_title} - {episode_title}")

    # Sanitize filenames (replace spaces and slashes)
    safe_title = episode_title.replace(" ", "_").replace("/", "_")
    safe_podcast = podcast_title.replace(" ", "_").replace("/", "_")
    filename_base = f"{safe_podcast}_{safe_title}"

    audio_path = AUDIO_DIR / f"{filename_base}.mp3"
    transcript_path = TRANSCRIPT_DIR / f"{filename_base}.json"

    # Skip if transcript already exists
    if transcript_path.exists():
        print(f"Transcript already exists: {transcript_path}")
        continue

    # Step 1: Download audio if not present
    if not audio_path.exists():
        try:
            subprocess.run([
                "yt-dlp",
                "-x", "--audio-format", "mp3",
                "-o", str(audio_path),
                audio_url
            ], check=True)
        except subprocess.CalledProcessError as e:
            print(f"Failed to download audio: {e}")
            continue

    # Step 2: Transcribe audio file with Whisper
    try:
        result = model.transcribe(str(audio_path))
        transcript_text = result["text"]

        # Save only transcript text as JSON
        with open(transcript_path, "w", encoding="utf-8") as f:
            json.dump({"transcript": transcript_text}, f, indent=4, ensure_ascii=False)

        print(f"Transcript saved: {transcript_path}")
    except Exception as e:
        print(f"Whisper transcription error: {e}")
        continue



 Processing: Your Personal Revolution - Rewiring Unconscious Patterns for Authentic Change
[generic] Extracting URL: https://dts.podtrac.com/redirect.mp3/api.spreaker.com/download/episode/66312309/05_rewiring_uncon...authentic_change.mp3
[generic] 05_rewiring_unconscious_patterns_for_authentic_change: Downloading webpage
[redirect] Following redirect to https://d1bxy2pveef3fq.cloudfront.net/episodes/original/50441090?session_id=4634d2f5-8814-5a1c-849d-612d0a7efaad&ab=192&al=802533&ao=4096&cc=NYNNNNYYYY&ct=DOWNLOAD&episode_id=66312309&show_id=5510562&user_id=16429250&organization_id=11414499&tenant=SPREAKER&timestamp=1748792123&ppi=33f40470-bc29-54f7-965b-272901a16e68&epi=7a111a55-a966-5f11-b2d6-5969196eb7a5&media_type=static&Expires=1748878523&Key-Pair-Id=K1J2BR3INU6RYD&Signature=tftmj974Ep5~X75l3ipy4hVBuuIIz4TOuQCRyGsUSzzUB0cLHz-YByKTlVaLQ6XKTAYE0OHRd1FX2GbawocPKIYP1pMV9tRr9nlZtpUnzrmPmaclha-VyVmhU77YSTcTTel70DnRuJXj~20q8Oj7dJxXnu3zUgb9occelf64j9hwtOW1ryDjLwJVl3QvZcA3Ay1hN17MMU6cAEz8


ERROR: Interrupted by user


KeyboardInterrupt: 